## Jose Luis Padilla 

# Modulo 3 
# Caso práctico: Modelos de series temporales (datos de tráfico de la M30 de Madrid)

In [6]:
import pandas as pd
import numpy as np
import altair as alt
import os.path
import urllib.request

## Descripción del caso

En este caso vamos a realizar un análisis sencillo de SERIES TEMPORALES. Una característica muy destacable de las series temporales es la posibilidad de extraer varias *features* interesantes a partir del dato único del momento: puede ser relevante qué hora del día es, o qué dia de la semana, o qué mes del año, etc. En este ejemplo no realizaremos un modelado complejo ni predicciones a futuro, pero utilizaremos las mismas métricas que se usan para valorar esas predicciones para comparar distintos casos de indicadores de tráfico.

## Acceso a los datos y preparación

Vamos a leer los datos de tráfico en Madrid de la web datos.madrid.es, de manera análoga a como se hizo en el caso de la unidad 3.

(En este ejemplo, este medio de acceso a los datos está comentado, y se sustituye por un acceso local, para que sea posible ejecutar el notebook aún sin conexión a internet o aunque el sitio datos.madrid.es deje de estar disponible. De todos modos se muestra aquí con fines didácticos)

In [7]:
# # Si no existe el directorio 'data', lo creamos
# if not os.path.exists('./datos'):
#     os.makedirs('./datos')

# # URLs de los datos a acceder y nombre de los ficheros en los que guardarlos
# URL_DATOS_TRAFICO = 'https://datos.madrid.es/egob/catalogo/208627-78-transporte-ptomedida-historico.zip'
PATH_DATOS_TRAFICO = 'D:\\papi\\BIG DATA\MASTER\\Modulo 03\\Ejercicios\\trafico2019\\traficoNov2019.zip'

# # El ayuntamiento ofrece un pequeño documento con la descripción de los datos, que aprovechamos para descargar también
# URL_PDF_TRAFICO = 'https://datos.madrid.es/FWProjects/egob/Catalogo/Transporte/Trafico/ficheros/Estructura_DS_Contenido_Trafico_Historico.pdf'
# PATH_PDF_TRAFICO = './datos/traficoNov2019.pdf'

# if not os.path.exists(PATH_DATOS_TRAFICO):
#     urllib.request.urlretrieve(URL_DATOS_TRAFICO, PATH_DATOS_TRAFICO)
    
# if not os.path.exists(PATH_PDF_TRAFICO):
#     urllib.request.urlretrieve(URL_PDF_TRAFICO, PATH_PDF_TRAFICO)

In [8]:
trafico_raw = pd.read_csv(PATH_DATOS_TRAFICO, encoding = 'latin1', sep = ';', parse_dates = ['fecha'])
trafico_raw.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2019-11-01 00:00:00,M30,1320,3.0,0,50.0,N,5
1,1001,2019-11-01 00:15:00,M30,1260,3.0,0,54.0,N,5
2,1001,2019-11-01 00:30:00,M30,948,3.0,0,56.0,N,5
3,1001,2019-11-01 00:45:00,M30,888,3.0,0,59.0,N,5
4,1001,2019-11-01 01:00:00,M30,972,3.0,0,59.0,N,5


In [9]:
trafico_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11186837 entries, 0 to 11186836
Data columns (total 9 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id                   int64         
 1   fecha                datetime64[ns]
 2   tipo_elem            object        
 3   intensidad           int64         
 4   ocupacion            float64       
 5   carga                int64         
 6   vmed                 float64       
 7   error                object        
 8   periodo_integracion  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 768.1+ MB


In [10]:
# Fitramos los datos de interés para este análisis
trafico = trafico_raw[trafico_raw.tipo_elem == 'M30'][['id', 'fecha', 'intensidad', 'vmed']]

In [11]:
# Algunos de los puntos de medida no tienen la serie temporal completa; a efectos de este caso, nos 
# sirve con eliminar esos casos, ya que hay datos más que suficientes para el análisis
datos_completos = trafico.id.value_counts() == 2880
ids_datos_completos = datos_completos[datos_completos == True].index.to_list()

trafico = trafico[trafico.id.isin(ids_datos_completos)]
trafico.head(6)

,id,fecha,intensidad,vmed
0,1001,2019-11-01 00:00:00,1320,50.0
1,1001,2019-11-01 00:15:00,1260,54.0
2,1001,2019-11-01 00:30:00,948,56.0
3,1001,2019-11-01 00:45:00,888,59.0
4,1001,2019-11-01 01:00:00,972,59.0
5,1001,2019-11-01 01:15:00,768,62.0


In [12]:
# Vemos el contenido de nuestro DataFrame
trafico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354240 entries, 0 to 9644940
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          354240 non-null  int64         
 1   fecha       354240 non-null  datetime64[ns]
 2   intensidad  354240 non-null  int64         
 3   vmed        354240 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 13.5 MB


In [13]:
trafico.head(8)

,id,fecha,intensidad,vmed
0,1001,2019-11-01 00:00:00,1320,50.0
1,1001,2019-11-01 00:15:00,1260,54.0
2,1001,2019-11-01 00:30:00,948,56.0
3,1001,2019-11-01 00:45:00,888,59.0
4,1001,2019-11-01 01:00:00,972,59.0
5,1001,2019-11-01 01:15:00,768,62.0
6,1001,2019-11-01 01:30:00,588,66.0
7,1001,2019-11-01 01:45:00,696,66.0


## Visualización y análisis

En el caso de conjuntos de datos de un cierto tamaño, como es este, el uso de visualizaciones se hace prácticamente imprescindible para poder entender el contenido de los datos.

In [14]:
# Mostramos, mediante un sencillo gráfico, la intensidad y la velocidad a lo largo del mes en un
# punto concreto de la M30. Mediante los controles del ratón es posible aumentar la escala del gráfico
# para apreciar más detalle

data = trafico[trafico.id == 6890]

alt.Chart(data, width=900, height=150)\
        .mark_line()\
        .encode(x = 'fecha:T', y = alt.Y(alt.repeat("row"), type='quantitative'))\
        .repeat(row = ['intensidad', 'vmed'])\
        .interactive(bind_y = False)

alt.RepeatChart(...)

In [15]:
# Mostramos los mismos gráficos para otro punto de medida distinto

data = trafico[trafico.id == 6843]

alt.Chart(data, width=900, height=150)\
        .mark_line()\
        .encode(x = 'fecha:T', y = alt.Y(alt.repeat("row"), type='quantitative'))\
        .repeat(row = ['intensidad', 'vmed'])\
        .interactive(bind_y = False)

alt.RepeatChart(...)

Los gráficos anteriores nos permiten apreciar varias cosas. Hay una cierta relación entre la intensidad del tráfico y la velocidad media, pero ni mucho menos esa relación es directa. También pueden apreciarse patrones temporales claros: existe un componente horario, que hace que por la noches la intensidad sea mucho menor, y se aprecia también una diferencia entre distintos días de la semana.

## Métricas

Vamos a tratar de probar una hipótesis: que el patrón horario de la congestión del tráfico es distinto en distintos días de la semana. Para ello, haremos la media sobre todos los puntos de control del tráfico y para cada día de la semana, y comprararemos los resultados.

In [16]:
resumen = trafico.groupby([trafico.fecha.dt.weekday, trafico.fecha.dt.time])[['intensidad', 'vmed']].mean().rename_axis(['Dia', 'Hora']).reset_index()
resumen.Hora = resumen.Hora.astype('str')

In [17]:
alt.Chart(resumen, width=900, height=200)\
        .mark_line()\
        .encode(x = 'Hora:O', y = alt.Y(alt.repeat("row"), type='quantitative'), color = 'Dia:N')\
        .repeat(row = ['intensidad', 'vmed'])

alt.RepeatChart(...)

Se aprecia fácilmente como el sábado y el domingo (días de la semana 5 y 6) presentan un patrón horario muy diferente, y cómo el viernes difiere algo de los demás días de diario, con menor intensidad por la mañana, algo mayor alreadedor de las tres, y menor por la tarde (como corresponde a la costumbre, bastante común en Madrid, de que la semana laboral termine el viernes a las tres)

Para tratar de cuantificar esas diferencias, vamos a calcular algunas métricas

In [24]:
# Definimos unas sencillas funciones para calcular dos métricas tipicas de comparación de series temporales:
# MAE, mean absolute error
def MAE(serie1, serie2):
    return np.mean(np.abs(serie1 - serie2))
# RMSE, root mean squared error
def RMSE(serie1, serie2):
    return np.sqrt(np.mean(np.square(serie1 - serie2)))

In [25]:
# Compobamos que una serie es igual a sí misma

RMSE(resumen[resumen.Dia == 2].intensidad.to_numpy(), resumen[resumen.Dia == 2].intensidad.to_numpy())

0.0

In [27]:
# Comparamos la diferencia entre un lunes y un martes, y entre un lunes y un domingo
(RMSE(resumen[resumen.Dia == 0].intensidad.to_numpy(), resumen[resumen.Dia == 1].intensidad.to_numpy()) , 
RMSE(resumen[resumen.Dia == 0].intensidad.to_numpy(), resumen[resumen.Dia == 6].intensidad.to_numpy()))

(54.146123493799934, 642.5349748823916)

In [28]:
# Naturalmente, estas métricas y muchas más están disponibles en las librerías estándar de ciencia de datos,
# y de manera general es preferible usar las versiones de las librerías, por rendimiento, corrección, etc.
from sklearn.metrics import mean_squared_error

mean_squared_error(resumen[resumen.Dia == 0].intensidad.to_numpy(), resumen[resumen.Dia == 1].intensidad.to_numpy(), squared = False)

54.146123493799934

In [22]:
# Para hacer una comparación de todos los días entre sí, vamos a hacer una tabla cruzada...
series = {n: resumen[resumen.Dia == n].intensidad.to_numpy() for n in range(7)}

diffs_RMSE = pd.DataFrame([(i, j, RMSE(series[i], series[j])) for i in range(7) for j in range(7)], columns = [ 'Dia de referencia', 'Dia de comparación', 'RMSE'])

In [29]:
# Representamos el valor de la diferencia entre días mediante un heatmap
alt.Chart(diffs_RMSE, width=300, height=300).mark_rect()\
        .encode(x = 'Dia de referencia:O', y = 'Dia de comparación:O', color = 'RMSE:Q')

alt.Chart(...)